**MILESTONE-1**

In [ ]:
import easyocr
import cv2
import pandas as pd
from IPython.display import display

In [ ]:
csv_path = "diabetes.csv" 
df = pd.read_csv(csv_path)  

In [ ]:
print("Diabetes Dataset (First 5 Rows):")
display(df.head())

In [ ]:
# Load image
image_path = "medical_report.png"
image = cv2.imread(image_path)

In [ ]:
reader = easyocr.Reader(['en'])

In [ ]:
results = reader.readtext(image)

In [ ]:
print("\nExtracted Text from Medical Report:\n")
for bbox, text, confidence in results:
    print(f"{text} (Confidence: {confidence:.2f})")

**MILESTONE-2**

**XG BOOST**

In [ ]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
df = pd.read_csv('diabetes.csv')
print("Diabetes Dataset (First 5 Rows):")
display(df.head())

cols_to_fix = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
for col in cols_to_fix:
    median_val = df[df[col] != 0][col].median()
    df[col] = df[col].replace(0, median_val)

In [ ]:
X = df.drop('Outcome', axis=1)
y = df['Outcome']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

In [ ]:
model = XGBClassifier(
    n_estimators=100,
    max_depth=3,           
    learning_rate=0.05,    
    reg_alpha=1,           
    reg_lambda=1.5,       
    subsample=0.7,         
    colsample_bytree=0.7,  
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42
)

model.fit(X_train, y_train)

In [ ]:
train_preds = model.predict(X_train)
test_preds = model.predict(X_test)

print(f"Training Accuracy: {accuracy_score(y_train, train_preds) * 100:.2f}%")
print(f"Testing Accuracy:  {accuracy_score(y_test, test_preds) * 100:.2f}%")
print("\n--- Classification Report ---")
print(classification_report(y_test, test_preds))

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

style = {'description_width': 'initial'}

# Inputs for all 8 features
inputs = {
    'Pregnancies': widgets.IntText(value=0, description="Pregnancies:", style=style),
    'Glucose': widgets.IntText(value=100, description="Glucose (mg/dL):", style=style),
    'BloodPressure': widgets.IntText(value=70, description="Blood Pressure:", style=style),
    'SkinThickness': widgets.IntText(value=20, description="Skin Thickness:", style=style),
    'Insulin': widgets.IntText(value=80, description="Insulin:", style=style),
    'BMI': widgets.FloatText(value=25.0, description="BMI:", style=style),
    'DPF': widgets.FloatText(value=0.47, description="Pedigree Function:", style=style),
    'Age': widgets.IntText(value=30, description="Age:", style=style)
}

btn = widgets.Button(description="Check Alerts", button_style='warning')
output = widgets.Output()

#THRESHOLD ALERT FUNCTION
def check_thresholds(b):
    with output:
        clear_output()
        print("="*35)
        print("       MEDICAL RISK ALERTS      ")
        print("="*35)
        
        any_alert = False

        # Glucose Logic
        if inputs['Glucose'].value >= 126:
            print("❌ ALERT: High Glucose (Diabetic)")
            any_alert = True
        elif inputs['Glucose'].value >= 100:
            print("⚠️ WARNING: Elevated Glucose (Pre-diabetic)")
            any_alert = True

        # BMI Logic
        if inputs['BMI'].value >= 30:
            print("❌ ALERT: Obesity Detected")
            any_alert = True
        elif inputs['BMI'].value >= 25:
            print("⚠️ WARNING: Overweight")
            any_alert = True

        # Blood Pressure Logic
        if inputs['BloodPressure'].value >= 90:
            print("❌ ALERT: High Blood Pressure")
            any_alert = True

        if not any_alert:
            print("✅ Clinical values are within normal limits.")
        
        print("="*35)
        print("Note: Click again after updating values.")


btn.on_click(check_thresholds)
ui_layout = widgets.VBox(list(inputs.values()) + [btn])
display(ui_layout, output)

In [ ]:
dzcdv